# ENSF 544 Project Phase 1
Note: All comments for functions are explained in the markdown section before the code section

# Read pickle file into DataFrame
for bug reports and display the dataframe

In [12]:
import re, string
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem.snowball import SnowballStemmer
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

bug_reports = pd.read_pickle("allBugReports.pickle")
bug_reports

,fix,text,fixdate,summary,description,project,average_precision
id,,,,,,,
217,[org.apache.commons.collections.map.flat3map.j...,NaN,2006-07-18 22:02:11,Flat3Map.Entry.setValue() overwrites other Ent...,Flat3Map&amp;apos;s Entry objects will overwri...,COLLECTIONS,0.0
214,[org.apache.commons.collections.testextendedpr...,NaN,2006-07-18 22:44:33,ExtendedProperties - field include should be n...,"The field ""include"" in ExtendedProperties is c...",COLLECTIONS,0.0
222,[org.apache.commons.collections.testlistutils....,NaN,2006-08-18 19:01:22,CollectionUtils removeAll is actually retainAll,"The removeAll(Collection collection, Collectio...",COLLECTIONS,0.0
261,[org.apache.commons.collections.map.flat3map.j...,NaN,2007-08-20 14:11:54,Flat3Map.remove() does not return the correct ...,final Flat3Map m = new Flat3Map();\n ...,COLLECTIONS,0.0
264,[org.apache.commons.collections.fasttreemap.java],NaN,2007-08-31 09:39:59,FastTreeMap forgets the comparator,In line 359 and 582 of the current 3.2 release...,COLLECTIONS,0.0
...,...,...,...,...,...,...,...
12661,[org.springframework.core.annotation.annotated...,NaN,2015-01-24 09:20:46,@TestExecutionListeners is not fully supported...,Status Quo\nWhen using TestNG and deriving a t...,SPR,0.0
11598,[org.springframework.core.annotation.annotated...,NaN,2015-05-13 07:20:33,Favor more locally declared composed annotatio...,Background\nThis issue picks up where SPR-1147...,SPR,0.0
14080,[org.springframework.web.util.uricomponentsbui...,NaN,2016-03-24 13:17:04,DefaultCorsProcessor&apos;s origin comparison ...,This plays in part with SPR-13691 but is more ...,SPR,0.0


# Preparation for preprocessing

1.java_keywords: self-defined document that collects all keywords from java, this is used as the same way as ENGLISH_STOP_WORDS

2.camel_case_split(text): use regex to have the pattern that are going to splite the java variable and function names. (Due to the naming rules of java, for instance, setAlarm, theVariableName...)

In [13]:
import re
f = open('java_keywords.txt','r')
java_keywords = f.read().split()
stemmer = SnowballStemmer("english")

def camel_case_split(text):  
    return re.findall(r'[A-Za-z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', text)
      

clean_text(text): 1. replace all dot to space due to the naming rules of java (For instance: str.translate needs to be splited).2.remove all punctuation 3.split the text 4. eliminate all words belong to stop words and java_keywords 5. split camel case words 6. stem and append each word in to the list

In [14]:
def clean_text(text):
    
    # text: str, returns: str
    # make the text lowercase, remove all punctuations, split words into list if they are not stop word
    if text is None:
        return
    text = text.replace('.',' ')
    text = text.translate(str.maketrans(' ', ' ', string.punctuation))
    textlist = []
    for word in text.split():
        if word and word not in set(ENGLISH_STOP_WORDS) and word not in set(java_keywords):
            word = camel_case_split(word)
            if isinstance(word, list) == True:
                for i in word:
                    textlist.append(stemmer.stem(i))
            else:
                textlist.append(stemmer.stem(word))
    #textlist = [stemmer.stem(word) for word in text.split() if word and word not in set(ENGLISH_STOP_WORDS) and word not in set(java_keywords)]
    text = " ".join(textlist)
    return text

# Corpus for bug report
use clean_text function for summary and description columns and combine them into a new column "query"

In [15]:
# going over the summary and description columns
bug_reports['summary'] = bug_reports['summary'].map(clean_text)
bug_reports['description'] = bug_reports['description'].map(clean_text)
# combine two columns into new column query
bug_reports['query'] = bug_reports['summary']+bug_reports['description']

bug_reports

,fix,text,fixdate,summary,description,project,average_precision,query
id,,,,,,,,
217,[org.apache.commons.collections.map.flat3map.j...,NaN,2006-07-18 22:02:11,flat map entri set valu overwrit entri valu,flat mapampaposs entri object overwrit entryam...,COLLECTIONS,0.0,flat map entri set valu overwrit entri valufla...
214,[org.apache.commons.collections.testextendedpr...,NaN,2006-07-18 22:44:33,extend properti field includ nonstat,the field includ extend properti current insta...,COLLECTIONS,0.0,extend properti field includ nonstatthe field ...
222,[org.apache.commons.collections.testlistutils....,NaN,2006-08-18 19:01:22,collect util remov all actual retain all,the remov all collect collect collect remov me...,COLLECTIONS,0.0,collect util remov all actual retain allthe re...
261,[org.apache.commons.collections.map.flat3map.j...,NaN,2007-08-20 14:11:54,flat map remov doe correct valu size,flat map m flat map m integ integ m integ inte...,COLLECTIONS,0.0,flat map remov doe correct valu sizeflat map m...
264,[org.apache.commons.collections.fasttreemap.java],NaN,2007-08-31 09:39:59,fast tree map forget compar,in line current releas replac map tree map map...,COLLECTIONS,0.0,fast tree map forget comparin line current rel...
...,...,...,...,...,...,...,...,...
12661,[org.springframework.core.annotation.annotated...,NaN,2015-01-24 09:20:46,test execut listen fulli support metaannot,status quo when use test ng deriv test abstrac...,SPR,0.0,test execut listen fulli support metaannotstat...
11598,[org.springframework.core.annotation.annotated...,NaN,2015-05-13 07:20:33,favor local declar compos annot inherit annot,background this issu pick sp left status quo t...,SPR,0.0,favor local declar compos annot inherit annotb...
14080,[org.springframework.web.util.uricomponentsbui...,NaN,2016-03-24 13:17:04,default cor processoraposs origin comparison r...,this play sp natur default cor processor doe c...,SPR,0.0,default cor processoraposs origin comparison r...


# Read pickle file into DataFrame
for source code and display the dataframe

In [16]:
source_codes = pd.read_pickle("allSourceCodes.pickle")
source_codes

,filename,unprocessed_code,project
0,\gitrepo\src\java\org\apache\commons\collectio...,/*\n * Licensed to the Apache Software Founda...,COLLECTIONS
1,\gitrepo\src\java\org\apache\commons\collectio...,/*\n * Licensed to the Apache Software Founda...,COLLECTIONS
2,\gitrepo\src\java\org\apache\commons\collectio...,/*\n * Licensed to the Apache Software Founda...,COLLECTIONS
3,\gitrepo\src\java\org\apache\commons\collectio...,/*\n * Licensed to the Apache Software Founda...,COLLECTIONS
4,\gitrepo\src\java\org\apache\commons\collectio...,/*\n * Licensed to the Apache Software Founda...,COLLECTIONS
...,...,...,...
5893,\gitrepo\src\test\java\org\springframework\exp...,/*\n * Copyright 2002-2012 the original author...,SPR
5894,\gitrepo\src\test\java\org\springframework\exp...,/*\n * Copyright 2002-2012 the original author...,SPR
5895,\gitrepo\src\test\java\org\springframework\sch...,/*\n * Copyright 2002-2014 the original author...,SPR
5896,\gitrepo\src\test\java\org\springframework\tra...,/*\n * Copyright 2002-2014 the original author...,SPR


# Preparation for preprocessing
related functions for preprocess the filename

1.escape_special_characters(text): this function is used for preprocessing the file path, replace the special characters \x to \\x

2.clean_text_filename(text): this function is used for comparing between the filename of bug report dataframe and the filename of source code datafram, replace all \\ to .

In [17]:
def escape_special_characters(text):
    if text is None:
        return
    text = text.replace("\a", "\\a")
    text = text.replace("\b", "\\b")
    text = text.replace("\f", "\\f")
    text = text.replace("\n", "\\n")
    text = text.replace("\r", "\\r")
    text = text.replace("\t", "\\t")
    text = text.replace("\v", "\\v")
    return text
    
def clean_text_filename(text):
    if text is None:
        return
    text = escape_special_characters(text)
    #text = text.translate(str.maketrans('\\', '.'))
    text = text.replace("\\",".")
    return text

# Corpus for source file
1.use clean_text_filename to preprocess the data for filename column 

2.use clean_text function for unprocessed_code column and put them into a new column "processed_code"

In [18]:
# going over the unprocessed_code and save into processed_code column
source_codes['processed_code'] = source_codes['unprocessed_code'].map(clean_text)
source_codes['filename'] = source_codes['filename'].map(clean_text_filename)
source_codes

,filename,unprocessed_code,project,processed_code
0,.gitrepo.src.java.org.apache.commons.collectio...,/*\n * Licensed to the Apache Software Founda...,COLLECTIONS,licens apach softwar foundat asf contributor l...
1,.gitrepo.src.java.org.apache.commons.collectio...,/*\n * Licensed to the Apache Software Founda...,COLLECTIONS,licens apach softwar foundat asf contributor l...
2,.gitrepo.src.java.org.apache.commons.collectio...,/*\n * Licensed to the Apache Software Founda...,COLLECTIONS,licens apach softwar foundat asf contributor l...
3,.gitrepo.src.java.org.apache.commons.collectio...,/*\n * Licensed to the Apache Software Founda...,COLLECTIONS,licens apach softwar foundat asf contributor l...
4,.gitrepo.src.java.org.apache.commons.collectio...,/*\n * Licensed to the Apache Software Founda...,COLLECTIONS,licens apach softwar foundat asf contributor l...
...,...,...,...,...
5893,.gitrepo.src.test.java.org.springframework.exp...,/*\n * Copyright 2002-2012 the original author...,SPR,copyright origin author author licens apach li...
5894,.gitrepo.src.test.java.org.springframework.exp...,/*\n * Copyright 2002-2012 the original author...,SPR,copyright origin author author licens apach li...
5895,.gitrepo.src.test.java.org.springframework.sch...,/*\n * Copyright 2002-2014 the original author...,SPR,copyright origin author author licens apach li...
5896,.gitrepo.src.test.java.org.springframework.tra...,/*\n * Copyright 2002-2014 the original author...,SPR,copyright origin author author licens apach li...


# Compare the textual similarity


related functions for argsorting a list and only keep the top 1/5/10

In [19]:
def my_sort_top1(x):
    x = np.argsort(x)
    x = x[::-1]
    x = x[:1]
    return x
def my_sort_top5(x):
    x = np.argsort(x)
    x = x[::-1]
    x = x[:5]
    return x
def my_sort_top10(x):
    x = np.argsort(x)
    x = x[::-1]
    x = x[:10]
    return x

# Report result by using MAP
Related functions that are going to be used in calculating the MAP

1.recision_score_query(pred_filenames, true_filenames): steps for calculating the precision score for each query, argument1 is the top x  filenames that are predicted to be debugged from source code dataframe, argument2 is the actual fixed filenames from bug report dataframe. The function loop the pred_filenames and for each element, trying to find if it match one of the true_filenames. Save corepsonding value in the pred_value list and true_value list. Then calculating the precision score based on these two lists by using the precision_score function and set the argument average = 'micro'

2.avg_of_list(l): calculate the average of a number list

3.map_for_sims(sims_for_tops,select_bug_reports, select_source_codes): steps for calculating the mean average precision for average precision score from three top x. The function calculate the precision score for each query, then calculate the average precision score for each matrix. Finally, calculate the MAP value for each project

In [20]:
from sklearn.metrics import precision_score
def precision_score_query(pred_filenames, true_filenames):
      
    pred_value = [0] * len(true_filenames)
    index = 0
    for pred_filename in pred_filenames:
        for true_filename in true_filenames:
            if pred_filename.endswith(true_filename) == True:
                # print('index = ',index)
                if index >= len(true_filenames):
                    break
                    # raise Exception('!!Out!!')
                pred_value[index] = 1
                index = index + 1 
                break
    true_value = [1]*len(true_filenames)
    # print(true_value,pred_value)
    p_s = precision_score(true_value,pred_value,average='micro')
    # print(p_s)
    return p_s

def avg_of_list(l):
    total = sum(l)
    length = len(l)
    average = total/length
    return average

def map_for_sims(sims_for_tops,select_bug_reports, select_source_codes):
    for simiter in sims_for_tops:
        filename_for_src = select_source_codes['filename']

        pred_filenames = []
        true_filenames = select_bug_reports.iloc[0].at['fix']
        p_s_list_for_project = []
        for rows in simiter:
            for item in rows:
                pred_filenames.append(filename_for_src[item])
                p_s = precision_score_query(pred_filenames, true_filenames)
                p_s_list_for_project.append(p_s)
        aps_for_tops.append(avg_of_list(p_s_list_for_project))

    return avg_of_list(aps_for_tops)

# Mean steps for looping all 12 projects and get the individual MAP value
explanation for steps in this section: for each project, generate the similarity matrix by using a “naive” algorithm that uses ARPACK as an eigensolver on X * X.T or X.T * X, then generate three version of the matrix based on keeping the top x similarity source files. For each matrix, each row in the matrix means different query, calculate the precision score for each query, then calculate the average precision score for each matrix. Finally, calculate the average of three top x a_p_s to get the MAP value for each project

note: the print function code in this section help to output individual MAP value for each project

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
import numpy

projects = list(source_codes['project'].value_counts().index)
# print(projects)
map_for_projects = []
# looping the 12 projects
i = 0
for project in projects:
    # print(project)
    select_bug_reports = bug_reports.loc[bug_reports['project'] == project]
    select_source_codes =  source_codes.loc[source_codes['project'] == project]
    
    select_bug_reports = select_bug_reports.replace(np.nan, '', regex=True)
    
    bug_mat = CountVectorizer().fit_transform(select_bug_reports['query'])
    src_mat = CountVectorizer().fit_transform(select_source_codes['processed_code'])
    # fit lsa
    lsa = TruncatedSVD(2, algorithm = 'randomized')
    bug_lsa = lsa.fit_transform(bug_mat)
    src_lsa = lsa.fit_transform(src_mat)
    sim = np.asarray(numpy.asmatrix(bug_lsa) * numpy.asmatrix(src_lsa).T)
    

    sims_for_tops = []
    sims_for_tops.append(np.apply_along_axis(my_sort_top1, axis = 1, arr = sim))
    sims_for_tops.append(np.apply_along_axis(my_sort_top5, axis = 1, arr = sim))
    sims_for_tops.append(np.apply_along_axis(my_sort_top10, axis = 1, arr = sim))
    
    aps_for_tops = []
    map = map_for_sims(sims_for_tops,select_bug_reports, select_source_codes)
    map_for_projects.append(map)

# print(map_for_projects)
map_dict = {'Project':projects,'MAP':map_for_projects}
# map_dict

# Output the project and the coresponding MAP value

In [24]:
map_dict

{'Project': ['SPR',
  'SEC',
  'LDAP',
  'DATACMNS',
  'COLLECTIONS',
  'DATAMONGO',
  'DATAREST',
  'SOCIALFB',
  'LANG',
  'CONFIGURATION',
  'IO',
  'ELY'],
 'MAP': [0.0,
  0.0,
  0.6537735849056604,
  0.26160337552742713,
  0.32916666666666666,
  0.0,
  0.0,
  0.0,
  0.0,
  0.318796992481203,
  0.2716117216117216,
  0.0]}

In [23]:
pd.DataFrame.from_dict(data = map_dict)

,Project,MAP
0,SPR,0.000000
1,SEC,0.000000
2,LDAP,0.653774
3,DATACMNS,0.261603
4,COLLECTIONS,0.329167
5,DATAMONGO,0.000000
6,DATAREST,0.000000
7,SOCIALFB,0.000000
8,LANG,0.000000
9,CONFIGURATION,0.318797
